# Inför nästa labb

Collaborative filterint eller content filtering.  

Vid **content filtering** måste användaren ha någon sorts profil där den registrerar t ex reviews och sen så kan profilen matchas gentemot specifika spel exempelvis. Då kan den i praktiken försöka hitta andra profiler eller något annat mönster. Content filtering ger bra resultat, vanligt i ehandel och lite mer involverad. Problemet är att en användarbas måste byggas upp först innan metoden börjar att fungera ordentligt.  

I **collaborative filtering** försöker vi hitta likheter mellan t ex olika spel. Bygga upp en feature mängd där saker får höga vikter/värden om de är lika varandra. 

In [182]:
import pandas as pd 

# df = pd.read_csv("../data/steam-200k.csv", names=["user_id", "game_title", "activity", "game_time", "0"])
# df

Här måste vi bygga upp en feature mängd för att underlätta vår ML modell. All data måste vektoriseras.  

En sak att testa är kanske att räkna ut medelspeltiden för varje spelare, spel eller för alla spelare etc för att kunna avgöra när något spelas mer än genomsnittet av en person, spel vs spel eller alla spelare. Gruppera efter users, spel. Standardisera och normalisera värdena för speltid.  



In [183]:
# df = pd.read_csv("../data/movielens/movies.csv")
# df

här skulle t ex one-hot encoding vara bra på genrer, då får vi redan någon form av vektor. 

In [184]:
# df = pd.read_csv("../data/movielens/ratings.csv")
# df

Något att tänka på här kan vara att vissa användare kanske alltid recenserar högt och att vi då skulle behöva vikta recensionerna. Vi får dock göra ett avvägande på hur mycket vi vinner på att göra det. Vi vill ju inte veta hur bra filmen var eller inte bara om vi ska rekommendera den. 

In [185]:
# df = pd.read_csv("../data/movielens/tags.csv")
# df

Det här är alltså nyckelord som användare själva har matat in. Väldigt svår att använda men oerhört rik på information. Vi skulle kunna parsa texterna. Går att bygga upp någon form av tag cloud och hitta liknande filmer utifrån taggar. 

Men det enklaste skulle nog vara genrer och ratings. Det går också att göra någon form av content filtering men komplext eftersom vi måste extrahera all data per användare och bygga upp en databas av det. Så collaborative filtering är nog mycket enklare här. Ni kommer att få ganska så olika resultat. Labbens föreskrivna metod är helt enkelt KNN men för högre betyg kolla på andra också.  

Timestamps kan vara intressant för att se mönster: filmer för en viss tid på året, användaren över tid etc. Men för vårt fall kommer vi kanske inte att titta på timestamps.  

I och med att vi har länkar så kan ett program byggas som skrapar vissa sidor (information retrieval). Hämta t ex skådespelare. Det är något för nästkommande kurser dock.  

Det finns mycket information om hur du kan göra och använda olika modeller etc online. Många använder just detta datasetet. 

I denna labb ska vi inte skriva en rapport utan en applikation som rekommenderar filmer. (streamlit som Martin använde kanske??)  

Så som vi skulle vilja ha det är att vi matar in en film (movieID) och sen spottar den ut en rankad lista med rekommendationer. Vi behöver alltså ett värde på varje film (0 -> 1) hur bra rekommendation det är. I stället för soft-max och rekommendera bara en av dem så vill vi printa listan på de fem bästa. Det är ett sätt att göra det på och skulle kunna göras med bara ratings. KNN eller SVM kanske.  

Skulle vi vilja använda tags behöver vi göra ett moln (vektorisera taggarna). Det vi ser är ju basically en bag-of-words. Går ju att göra en TF-IDF. Ett dokument är alla tags som en viss användare har lagt. Då går det att jämföra taggar mellan två olika filmer. Går t ex att få en kosinuslikhet (först standardnormalisera, sen tar vi bara skalärprodukten mellan dem - pga standardnormalt så blir det cosinus likehet).  

Det går även att tänka sig en logistisk regression (multinomiellt klassifikationsproblem). När vi kodar data så finns det relation mellan kategorierna (lägsta kategorin är mindre än den högsta), vi vill i detta fall - till skillnad från i vanliga fall - att de ska vara rangordnade. Så en linjär diskriminant (LDA eller QDA) eller SVM med linear kernel och rangordnar filmerna. Här går ju att göra en predictproba för att få ut en sannolikhet över klasser.   

En värderegression skulle kanske gå. FÖrsöka predicta vad en användare skulle gilla för film som de inte sett utifrån de som den har sett. Vi får väl se hur bra det blir men med ratings, taggar och genrer kan vi kanske göra det.  



## Geeks for Geeks tutorial

In [186]:
users = pd.read_csv("../data/geeks4geeks/Users.csv")
users

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
...,...,...,...
278853,278854,"portland, oregon, usa",NaN
278854,278855,"tacoma, washington, united kingdom",50.0
278855,278856,"brampton, ontario, canada",NaN
278856,278857,"knoxville, tennessee, usa",NaN


In [187]:
ratings = pd.read_csv("../data/geeks4geeks/Ratings.csv")
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [188]:
books = pd.read_csv("../data/geeks4geeks/Books.csv")
books = books.drop_duplicates(subset="Book-Title").dropna(how="any").drop(columns=["Image-URL-S", "Image-URL-M", "Image-URL-L"], axis=1)
books.info()

/tmp/ipykernel_6378/271540461.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("../data/geeks4geeks/Books.csv")


<class 'pandas.core.frame.DataFrame'>
Index: 242129 entries, 0 to 271359
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   ISBN                 242129 non-null  object
 1   Book-Title           242129 non-null  object
 2   Book-Author          242129 non-null  object
 3   Year-Of-Publication  242129 non-null  object
 4   Publisher            242129 non-null  object
dtypes: object(5)
memory usage: 11.1+ MB


In [189]:
ratings = ratings.merge(books, on="ISBN").drop(columns="ISBN", axis=1)
ratings

,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,276725,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books
1,276729,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press
2,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press
3,276746,0,Lightning,Dean R. Koontz,1996,Berkley Publishing Group
4,276746,0,Manhattan Hunt Club,JOHN SAUL,2002,Ballantine Books
...,...,...,...,...,...,...
883067,276704,0,Edgar Cayce on the Akashic Records: The Book o...,Kevin J. Todeschi,1998,A.R.E. Press (Association of Research &amp; Enlig
883068,276704,9,Get Clark Smart : The Ultimate Guide for the S...,Clark Howard,2000,Longstreet Press
883069,276706,0,Eight Weeks to Optimum Health: A Proven Progra...,Andrew Weil,1997,Alfred A. Knopf
883070,276709,10,The Sherbrooke Bride (Bride Trilogy (Paperback)),Catherine Coulter,1996,Jove Books


[här gick streamen ner]

In [190]:
ratings.isna().sum()
ratings = ratings.dropna(how="any")

In [191]:
ratings[ratings["Book-Title"] == "1984"]

,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
2784,278158,0,1984,George Orwell,1990,Signet Book
7881,254,9,1984,George Orwell,1990,Signet Book
10410,1706,10,1984,George Orwell,1990,Signet Book
11659,2179,8,1984,George Orwell,1990,Signet Book
11722,2198,10,1984,George Orwell,1990,Signet Book
...,...,...,...,...,...,...
855725,267184,10,1984,George Orwell,1990,Signet Book
870375,271705,10,1984,George Orwell,1990,Signet Book
873982,273976,7,1984,George Orwell,1990,Signet Book
879819,275520,10,1984,George Orwell,1990,Signet Book


[jag har fler värden än R, de måste gjort något mer när streamen la ner]  

Något som kan vara värt att tänka på här är användare som lämnar många betyg, de brukar vara mer pålitliga medan de andra har högre bias (ett antagande). 

In [192]:
X = ratings.groupby("User-ID").count()["Book-Rating"] > 100
experts = X[X].index
ratings = ratings[ratings["User-ID"].isin(experts)]

y = ratings.groupby("Book-Title").count()["Book-Rating"] >= 50
classics = y[y].index
ratings = ratings[ratings["Book-Title"].isin(classics)]
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51997 entries, 990 to 882915
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   User-ID              51997 non-null  int64 
 1   Book-Rating          51997 non-null  int64 
 2   Book-Title           51997 non-null  object
 3   Book-Author          51997 non-null  object
 4   Year-Of-Publication  51997 non-null  object
 5   Publisher            51997 non-null  object
dtypes: int64(2), object(4)
memory usage: 2.8+ MB


In [193]:
design_matrix = ratings.pivot_table(index="Book-Title", columns="User-ID", values="Book-Rating")
design_matrix.fillna(0, inplace=True)
design_matrix

User-ID,254,507,882,1424,1435,1733,1903,2033,2110,2276,...,274808,275020,275970,276680,277427,277478,277639,278137,278188,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Nu har vi vektoriserat antalet användare (ANDERS: snarare varje bok?). Varje titel är en vektor av alla användarnas betyg. Betygen är däremot en godtycklig, rangordnad skala. Bra att skalera. 

In [194]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

scaler = StandardScaler(with_mean=True, with_std=True)
design_matrix_scaled = scaler.fit_transform(design_matrix)


In [195]:
similarity_score = cosine_similarity(design_matrix_scaled)
similarity_score

array([[ 1.        , -0.01859849,  0.08733311, ...,  0.01350255,
        -0.02201299, -0.02261981],
       [-0.01859849,  1.        , -0.0294832 , ..., -0.02238366,
        -0.03070822,  0.10549892],
       [ 0.08733311, -0.0294832 ,  1.        , ...,  0.00231518,
         0.00643198, -0.01337545],
       ...,
       [ 0.01350255, -0.02238366,  0.00231518, ...,  1.        ,
        -0.00500822, -0.01371862],
       [-0.02201299, -0.03070822,  0.00643198, ..., -0.00500822,
         1.        ,  0.00914464],
       [-0.02261981,  0.10549892, -0.01337545, ..., -0.01371862,
         0.00914464,  1.        ]], shape=(620, 620))

Ovan data är direkt användbar för ML. Detta är typ en KNN fast på ett märkligt sätt. 

In [196]:
import numpy as np

def recommend(book_title):
    index = np.where(design_matrix.index==book_title)[0][0]
    similar_books = sorted(list(enumerate(similarity_score[index])), key=lambda x: x[1], reverse=True)[1:6]
    data = []
    for index , similarity in similar_books:
        item = []
        temp_df = books[books["Book-Title"] == design_matrix.index[index]]
        item.extend(temp_df["Book-Title"].values)
        item.extend(temp_df["Book-Author"].values)
        data.append(item)
    return data

In [197]:
rec = recommend("1984")
rec

[["Foucault's Pendulum", 'Umberto Eco'],
 ['Tis : A Memoir', 'Frank McCourt'],
 ['Animal Farm', 'George Orwell'],
 ['The Glass Lake', 'Maeve Binchy'],
 ['Summer Pleasures', 'Nora Roberts']]